# Segment and cluster Toronto neighbourhoods
### Author: Kazimierz Hermaszewski
This notebook does the following:
- Scrape neighbourhood data from Wikipedia
- Geocode the neigbourhoods using the geopy library
- Explore the dataset and cluster the neighbourhoods

## Scrape data from Wikipedia

In [2]:
# install beautiful soup to scrape webpages
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeableWARNING: You are using pip version 21.0; however, version 21.1.1 is available.
You should consider upgrading via the 'c:\program files (x86)\python38-32\python.exe -m pip install --upgrade pip' command.



In [9]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [25]:
#pc_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood']) # initialise dataframe with columns

# get webpage content
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
url = 'http://invalid'
try:
    response = requests.get(url)
except requests.exceptions.RequestException as e:
    print(f'Error: {e}')

print(response.status_code)

soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)


Error: HTTPConnectionPool(host='invalid', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x164930B8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
200


In [ ]:


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


# shape of dataframe
df.shape